# Dates With Python
> Dealing with custom date intervals
- toc: true
- badges: true
- comments: true 
- author: Dan Hosanee
- categories: [python]
- image: images/pythonwithdates.jpg

# About

When you are tasked with custom date intervals such as last Friday of each month. 

## Sources 

Looking at [stack overflow](https://stackoverflow.com/questions/52721690/calculate-last-friday-of-month-in-pandas) we see a few solutions but the ideal answer for me uses the **date_range** method within pandas.

- The solution uses a DataFrame to get the weekly Fridays based on the source input
- Then uses the .groupby method on year and month selecting the last value

In [1]:
#collapse-output
import pandas as pd
from pandas.testing import assert_frame_equal

# Source Input 
df = pd.DataFrame({'Date' : pd.date_range(start=pd.to_datetime('2022-01-01'), end=pd.to_datetime('2022-12-31'), freq='D')})

# Create DataFrame with all Fridays in the daterange for min and max of df.Date
fridays = pd.DataFrame({'datetime': pd.date_range(df.Date.min(), df.Date.max(), freq='W-FRI')})

# use groubpy and last, to get the last Friday of each month into a list             
last_fridays_in_daterange = fridays.groupby([fridays.datetime.dt.year, fridays.datetime.dt.month]).last()['datetime'].tolist()

# find the data for the last Friday of the month
df[df.Date.isin(last_fridays_in_daterange)]

,Date
27,2022-01-28
55,2022-02-25
83,2022-03-25
118,2022-04-29
146,2022-05-27
174,2022-06-24
209,2022-07-29
237,2022-08-26
272,2022-09-30
300,2022-10-28


## Using Pandas Offset Feature 

The pandas documentations has a lot of custom date increments within the [Date offsets section](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects)

Following the above if we implement the offsets methods we can achieve the same result without needing to group by. 

In [2]:
#collapse-output
LastWeekOfMonthFri = pd.date_range(df.Date.min(),df.Date.max(),freq=pd.offsets.LastWeekOfMonth(weekday=4))

df[df.Date.isin(LastWeekOfMonthFri)]

,Date
27,2022-01-28
55,2022-02-25
83,2022-03-25
118,2022-04-29
146,2022-05-27
174,2022-06-24
209,2022-07-29
237,2022-08-26
272,2022-09-30
300,2022-10-28


Reconcile the two methods

In [3]:
assert_frame_equal(
        df[df.Date.isin(last_fridays_in_daterange)]
       ,df[df.Date.isin(LastWeekOfMonthFri)]
)

## Combine with pd.map

Using the pd.offsets within map method were able filter or create a new column within the df easily.

In [4]:
#collapse-output
LastWeekOfMonthFriMap = df.Date.map(pd.offsets.LastWeekOfMonth(weekday=4).rollforward)
df[df.Date.isin(LastWeekOfMonthFriMap.unique())]

,Date
27,2022-01-28
55,2022-02-25
83,2022-03-25
118,2022-04-29
146,2022-05-27
174,2022-06-24
209,2022-07-29
237,2022-08-26
272,2022-09-30
300,2022-10-28


Note that the rollforward method will shift/roll the date down when the date is not an offset
- Hence when the Date is the 28-01-2022 Offset will equal 28-01-2022 Rather than 25-02-2022
- Something to keep in mind when aggregating values

In [5]:
#collapse-output
LastWeekOfMonthFriMap = df.Date.map(pd.offsets.LastWeekOfMonth(weekday=4))
df['LastWeekOfMonthFriMap'] = LastWeekOfMonthFriMap

LastWeekOfMonthFriMapRollFwd = df.Date.map(pd.offsets.LastWeekOfMonth(weekday=4).rollforward)
df['LastWeekOfMonthFriMapRollFwd'] = LastWeekOfMonthFriMapRollFwd

df[df.LastWeekOfMonthFriMap!=LastWeekOfMonthFriMapRollFwd].head()

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/core/base.py:880: FutureWarning: DateOffset.__call__ is deprecated and will be removed in a future version.  Use `offset + other` instead.
  new_values = map_f(values, mapper)


,Date,LastWeekOfMonthFriMap,LastWeekOfMonthFriMapRollFwd
27,2022-01-28,2022-02-25,2022-01-28
55,2022-02-25,2022-03-25,2022-02-25
83,2022-03-25,2022-04-29,2022-03-25
118,2022-04-29,2022-05-27,2022-04-29
146,2022-05-27,2022-06-24,2022-05-27
